### Transferring motion-corrected data to Warp for sub-tomogram extraction

If Warp is used, please cite:

1. Tegunov, D., Cramer, P. Real-time cryo-electron microscopy data preprocessing with Warp. Nat Methods 16, 1146–1152 (2019). https://doi.org/10.1038/s41592-019-0580-y

For more information, please see http://www.warpem.com/warp/?page_id=378.

Motion-corrected images from Ot2Rec can be imported into Warp before CTF correction to join the sub-tomogram extraction pipeline. The motion-corrected images are a special case of "movie frames" where there is only one movie frame - the averaged motion-corrected frames. Please ensure that motion correction is disabled in Warp if you choose this route.

The parameters that are required are as follows:

#### Input

Note on unbinned tomogram dimensions:
- x, y dimensions determined from x- and y-dimensions of the motion corrected images
- z dimensions determined from -VolZ flag input into AreTomo

If either AreTomo or MotionCor was not used, Ot2Rec might not be able to give a full recommendation of the unbinned tomogram dimensions. You can find these dimensions manually - they should be the full unbinned x-y dimension of the images that go into Warp (usually size of the raw images) and the z-dimension is the thickness of the tomogram, which should be the z-dimension of the reconstructed tomogram used for particle picking.

In [ ]:
if os.path.isfile(f"{proj_name}_mc2.yaml"):
    with open(f"{proj_name}_mc2.yaml", "r") as f:
        mc2_config = yaml.load(f, Loader=yaml.FullLoader)

    # Print output directory
    print(
        f"Output directory for motion-corrected images: {mc2_config['System']['output_path']}"
    )

    # Get pixel size used in motioncor
    print(f"Output pixel size: {mc2_config['MC2']['desired_pixel_size']} Angstroms")

    # Get unbinned tomogram dimensions recommendations
    # xy determined from size of motion corrected images
    # z determined from VolZ flag of AreTomo
    if os.path.isfile(f"{proj_name}_mc2_mdout.yaml"):
        with open(f"{proj_name}_mc2_mdout.yaml", "r") as f:
            mc2_mdout = yaml.load(f, Loader=yaml.FullLoader)
        test_img = list(mc2_mdout["output"].values())[0]
        with mrcfile.open(test_img) as mrc:
            nx = int(mrc.header.nx)
            ny = int(mrc.header.ny)
        print(f"Unbinned tomogram dimensions x: {nx} px")
        print(f"Unbinned tomogram dimensions y: {ny} px")
    else:
        print("Motioncor2 mdout yaml file not found, cannot recommend xy dimensions.")

    if os.path.isfile(f"{proj_name}_aretomo_mdout.yaml"):
        with open(f"{proj_name}_aretomo_mdout.yaml", "r") as f:
            aretomo_mdout = yaml.load(f, Loader=yaml.FullLoader)
        cmd = list(aretomo_mdout["aretomo_cmd"].values())[0]
        if "-VolZ" not in cmd:
            print("Was AreTomo reconstruction done? VolZ not found in AreTomo cmd")
        else:
            volz = cmd.split("-VolZ ")[1].split()[0]
            print(f"Unbinned tomogram dimensions z: {volz} px")
    else:
        print("Aretomo mdout yaml file not found, cannot recommend z dimension")

else:
    print("Motioncor2 yaml file not found - cannot retrieve information")

#### Pre-processing

In [ ]:
# Get motioncor2 gain processing details
if os.path.isfile(f"{proj_name}_mc2.yaml"):
    with open(f"{proj_name}_mc2.yaml", "r") as f:
        mc2_config = yaml.load(f, Loader=yaml.FullLoader)
    gain = mc2_config["MC2"]["gain_reference"]

print(f"Gain correction performed with: {gain}")

#### CTF

In [ ]:
# Voltage

# Spherical aberration